#### Zadanie:
Vytvorte program na čistenie dát od nepotrebných dát:

- upravte dátový súbor identifikovaním premenných (IP, Cookie, user, DTime, RequestMethod, URL, StatusCode, Referrer, Agent),

- očistite dáta (RequestMethod/Version, StatusCode, URL).

Odovzdajte zdrojový kód programu + očistený logovací súbor (v jednom zip).

Príkazy v Pythone vhodné pre riešenie tohto zadania:

import pandas as pd
pd.read_csv() - načítanie log súboru

In [1]:
import json
import re
import pandas as pd

# https://pypi.org/project/apachelogs/
from apachelogs import LogParser
from urllib.parse import urlparse

In [2]:
extensions_not_allowed = ["css","js","flv","ico","swf","rss","xml","cur",
                          "json","svg","ttf","otf","woff","woff2","eot",
                          "jpg","bmp","jpg","jpeg","png","gif", "rss",]

http_methods_not_allowed = ["POST", "PUT", "HEAD", "DELETE", "PATCH", "OPTIONS"]

# 1xx, 4xx, 5xx
http_status_code_not_allowed = ("1", "4", "5")

In [3]:
# feed LogParser with correct values from Apache log file
parser = LogParser("%h %l %u %t \"%r\" %>s %b \"%{Referer}i\" \"%{User-Agent}i\"")

In [4]:
# open log file
with open("access2.log",  encoding='utf-8') as fp:
    processed = 0
    ref = ''
    agent = ''
    
    # preparing the empty arrays which will hold temp data for the pandas data work latter
    remote_host = []
    remote_logname = []
    remote_user = []
    request_time = []
    request_line_method = []
    request_line_uri = []
    request_line_protocol = []
    final_status = []
    bytes_sent = []
    referer = []
    user_agent = []
    
    quotation_marks_removed= []
    cleared = ''


    for entry in parser.parse_lines(fp):
        processed += 1
        line = entry.entry
        http_status_code = str(entry.final_status)
        try:
            method, uri, blob = entry.request_line.split()
        except ValueError:
            print(processed, entry.request_line)
            continue

        # here I checked method
        if method.upper() in http_methods_not_allowed:
            continue

        # here I checked http status code
        if http_status_code.startswith(http_status_code_not_allowed):
            continue
            
        # here I checked file extensions
        filename = urlparse(uri).path.split("/")[-1:][0]
        file_extension = filename.split(".")[-1:][0].lower()
        if file_extension in extensions_not_allowed:
            continue

        # populating cleaned data to arrays
        remote_host.append(entry.remote_host)
        remote_logname.append(entry.remote_logname)
        remote_user.append(entry.remote_user)
        request_time.append(entry.request_time)
        request_line_method.append(method)
        request_line_uri.append(uri)
        request_line_protocol.append(blob)
        final_status.append(entry.final_status)
        bytes_sent.append(entry.bytes_sent)

        referer.append(entry.headers_in["Referer"])
        user_agent.append(entry.headers_in["User-Agent"])
      
        

In [5]:
# creating the dictionary 
preprocessed_log_data_table = {
    "Remote Host": remote_host,
    "Remote Logname": remote_logname,
    "Remote User": remote_user,
    "Request Time": request_time,
    "Request Method": request_line_method,
    "Request Uri": request_line_uri,
    "Request Protocol": request_line_protocol,
    "Final Status Code": final_status,
    "Bytes Sent": bytes_sent,
    "Referer": referer,
    "User-Agent": user_agent,
    
}
# transforming the cleaned data to the pandas dataframe
log_data_table=pd.DataFrame(preprocessed_log_data_table)

In [6]:
log_data_table.to_csv(r'1_Čistenie_dát_I.csv',index=False,escapechar=";")